In [1]:
from scipy.stats import qmc
from modcma import ModularCMAES, Parameters
import ioh
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import sympy

In [2]:
problem_id = 3
problem_instance = 1
dimension = 4
# lambdas = [20, 20, 20, 20, 20]
# lambdas = [25, 25, 25, 25]
lambdas = [33,33,33]
# lambdas = [50, 50]
budget = int(dimension * 1e4)
iterations = 1000

In [3]:
problem = ioh.get_problem(fid=problem_id, instance=problem_instance, dimension=dimension, problem_type=ioh.ProblemType.BBOB)
CMAESs = []

In [6]:
# x0 = np.float64(qmc.scale(qmc.LatinHypercube(d=problem.meta_data.n_variables).random(n=len(lambdas)), l_bounds=problem.bounds.lb, u_bounds=problem.bounds.ub))#.reshape((len(lambdas), problem.meta_data.n_variables, 1))
x0 = np.random.uniform(-5, 5, (len(lambdas), dimension))
# x0 = points = np.array([[-4, -4], [-4, 0], [-4, 4], [0, -4], [0, 0], [0, 4], [4, -4], [4, 0], [4, 4]])
# x0 = points = np.array([[-4, -4], [-4, 4], [4, -4], [4, 4]])
labels = [n for n in range(1, len(lambdas)+1)]
print(x0, labels)

[[ 1.07446367 -3.76496802  0.37356156  4.10808921]
 [-1.844852   -3.7036209   4.18713062 -4.36709307]
 [ 4.28945162  0.81012235  1.33831519 -1.97803854]] [1, 2, 3]


In [5]:
x0.shape, x0.T

((3, 4),
 array([[-4.93226699,  1.05033961,  4.28043366],
        [ 0.11844398,  2.97817144, -3.90274651],
        [ 3.5705001 , -1.19071477,  0.39890047],
        [-2.79480057,  0.62435591,  3.80772633]]))

In [7]:
svm = SVC(kernel='linear').fit(X=x0, y=labels)

In [9]:
svm.coef_, svm.intercept_

(array([[ 0.06152521, -0.0012929 , -0.0803718 ,  0.17861631],
        [-0.09286599, -0.13215299, -0.02786723,  0.1757998 ],
        [-0.170808  , -0.12568394,  0.07932448, -0.06652257]]),
 array([ 0.22527772, -0.10956184, -0.40325335]))

In [ ]:
x0 = x0.reshape((len(lambdas), problem.meta_data.n_variables, 1))
print(x0.shape, x0)
for i in range(len(lambdas)):
    # print(lambdas[i], lambdas[i], x0[i], budget)
    CMAESs.append(
        ModularCMAES(
            problem,
            parameters=Parameters(
                d=problem.meta_data.n_variables,
                lambda_=lambdas[i],
                mu=lambdas[i],
                x0=x0[i],
                sigma0=0.2,
                budget=budget,
                initialization_correction="voronoi",
                bound_correction='mirror',
                area_corr=svm,
                subpopulation_target=labels[i]
            ),
        )
    )


In [ ]:
centroids_track = []

In [ ]:
centroids = np.float64([CMAESs[n].parameters.m.reshape((2,)) for n in range(len(lambdas))])
centroids

In [ ]:
def closest_point(coefs: np.ndarray, centroid: np.ndarray, new_point: np.ndarray):
    S = symbols([l for l,_ in zip(ascii_letters, range(len(centroid)))])
    St = S
    T = symbols('T')
    St.append(T)
    f = 0
    intersections, distances = [], []
    for coef, intercept in coefs:
        f = 0
        for s, v in zip(St, coef):
            print(s, v)
            f += v*s
        f = Eq((f), intercept)
        # solve for vector
        t = centroid - new_point
        t_functions = [f]
        for i in range(len(t)):
            f_t = Eq((new_point[i] + T*t[i]), S[i])
            t_functions.append(f_t)
        print(t_functions)
        values = solve((t_functions), tuple(St))
        print(values)
        intersection = np.float64([values[k] for k in S])
        intersections.append(intersection)
        distances.append(np.linalg.norm(centroid - intersection))
    j = np.argmin(distances)
    plane = coefs[j]
    intersection = intersections[j]
    vector = new_point - intersection

    # perform projection
    pj = -np.dot(plane, vector) / np.dot(plane, plane)
    pj_point = plane * pj + new_point

In [ ]:
voronoi = Voronoi(centroids)
print(voronoi.point_region)

for n in range(len(lambdas)):
    CMAESs[n].parameters.area_corr = voronoi
    CMAESs[n].parameters.subpopulation_target = voronoi.point_region[n]

for n in range(len(lambdas)):
    CMAESs[n].step()

In [ ]:
temp = np.float64([CMAESs[n].parameters.population.x for n in range(len(lambdas))])
sub_pops_points = [None]*len(lambdas)
for n in range(len(lambdas)):
    sub_pops_points[n] = temp[n].T
fig = voronoi_plot_2d(voronoi, show_points=False)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.scatter(x0[0,0], x0[0,1], label='1')
plt.scatter(x0[1,0], x0[1,1], label='2')
plt.scatter(x0[2,0], x0[2,1], label='3')
plt.scatter(x0[3,0], x0[3,1], label='4')
# plt.scatter(test_point[0], test_point[1], label='test')
for n in range(len(lambdas)):
    plt.scatter(sub_pops_points[n][:,0], sub_pops_points[n][:,1])
plt.legend()
plt.show()

In [ ]:
for _ in range(999):
    print(_)
    voronoi = Voronoi(centroids)
    print(voronoi.point_region)

    for n in range(len(lambdas)):
        CMAESs[n].parameters.area_corr = voronoi
        CMAESs[n].parameters.subpopulation_target = voronoi.point_region[n]

    for n in range(len(lambdas)):
        CMAESs[n].step()

In [ ]:
temp = np.float64([CMAESs[n].parameters.population.x for n in range(len(lambdas))])
sub_pops_points = [None]*len(lambdas)
for n in range(len(lambdas)):
    sub_pops_points[n] = temp[n].T
fig = voronoi_plot_2d(voronoi, show_points=False)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.scatter(x0[0,0], x0[0,1], label='1')
plt.scatter(x0[1,0], x0[1,1], label='2')
plt.scatter(x0[2,0], x0[2,1], label='3')
plt.scatter(x0[3,0], x0[3,1], label='4')
# plt.scatter(test_point[0], test_point[1], label='test')
for n in range(len(lambdas)):
    plt.scatter(sub_pops_points[n][:,0], sub_pops_points[n][:,1])
plt.legend()
plt.show()

In [ ]:
raise

In [ ]:
temp = np.float64([CMAESs[n].parameters.population.x for n in range(len(lambdas))])
sub_pops_points = [None]*len(lambdas)
for n in range(len(lambdas)):
    sub_pops_points[n] = temp[n].T
plt.xlim(-5, 5)
plt.ylim(-5, 5)
xx, yy = np.meshgrid(np.arange(-6, 6, .2), np.arange(-6, 6, .2))
Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape).astype(int)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
for n in range(len(lambdas)):
    plt.scatter(sub_pops_points[n][:,0], sub_pops_points[n][:,1], label=labels[n])
plt.scatter(centroids[:,0], centroids[:,1])
plt.legend()
plt.show()

In [ ]:
centroids = np.float64([CMAESs[n].parameters.m.reshape((2,)) for n in range(len(lambdas))])
print(centroids)
svc = svc.fit(centroids, labels)
for n in range(len(lambdas)):
    CMAESs[n].parameters.svc = svc
    CMAESs[n].step()

In [ ]:
temp = np.float64([CMAESs[n].parameters.population.x for n in range(len(lambdas))])
sub_pops_points = [None]*len(lambdas)
for n in range(len(lambdas)):
    sub_pops_points[n] = temp[n].T
plt.xlim(-5, 5)
plt.ylim(-5, 5)
xx, yy = np.meshgrid(np.arange(-6, 6, .2), np.arange(-6, 6, .2))
Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape).astype(int)
plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
for n in range(len(lambdas)):
    plt.scatter(sub_pops_points[n][:,0], sub_pops_points[n][:,1], label=labels[n])
plt.scatter(centroids[:,0], centroids[:,1])
plt.legend()
plt.show()

In [ ]:
raise ValueError

In [ ]:
for _ in range(1000):
    for j in range(len(CMAESs)):
        CMAESs[j].step()

In [ ]:
for cma in CMAESs:
    print(f"CMA: f:{cma.parameters.fopt}, {cma._fitness_func.optimum.y}, {cma.parameters.population}")